In [ ]:
from pydruid.db import connect

conn = connect(host='localhost', port=8888, path='/druid/v2/sql/', scheme='http')
curs = conn.cursor()

In [53]:
from string import Template
from typing import List

batting_stats_template = Template("""
with base_data as (
  select * from delivery_records WHERE __time > '$start_date' AND __time < '$end_date'  AND "match_type" = 'T20' $additional_filters
), dismissals as (
  select sum("is_wicket") as dismissal_count from base_data WHERE "wicket_player_id" = '$batter_id'
)
SELECT
  sum("batter_run_sum")*100.0/sum("delivery_count") as strike_rate,
  sum("batter_run_sum")*1.0/(SELECT sum(dismissal_count) from dismissals) as average
from base_data
where "batter_id" = '$batter_id'
""")


def get_venue_filter(venue_name:str):
    return 'venue_name=\'%s\''%venue_name

def get_adversary_filter(bowler_ids: List[str]):
    return 'bowler_id in (%s)'% ",".join(['\'%s\''%bowler_id for bowler_id in bowler_ids])

venue = 'Eden Gardens'
additional_filters ='and ' + get_venue_filter(venue)
additional_filters = ' and ' +  get_adversary_filter(['495d42a5'])
query = batting_stats_template.substitute(
    start_date = '2017-01-01',
    end_date='2023-01-01',
    additional_filters=additional_filters,
    batter_id='ba607b88')
print(query)

In [57]:

curs.execute(query)
for row in curs:
    print(row)

Row(strike_rate=110.0, average=33.0)


In [71]:

bowling_stats_template = Template("""
with base_data as (
  select * from delivery_records WHERE __time > '$start_date' AND __time < '$end_date'  AND "match_type" = 'T20' $additional_filters
), dismissals as (
  select sum("wicket_sum") as dismissal_count from base_data WHERE "bowler_id" = '$bowler_id'
)
SELECT
  sum("delivery_count")*1.0/(SELECT sum(dismissal_count) from dismissals) as strike_rate,
  sum("total_run_sum")*1.0/(SELECT sum(dismissal_count) from dismissals) as average
from base_data
where "bowler_id" = '$bowler_id'
""")


def get_venue_filter(venue_name:str):
    return 'venue_name=\'%s\''%venue_name

def get_adversary_filter(batter_ids: List[str]):
    return 'batter_id in (%s)'% ",".join(['\'%s\''%batter_id for batter_id in batter_ids])

additional_filters = ' and ' +  get_adversary_filter(['ba607b88'])
query = bowling_stats_template.substitute(
    start_date = '2017-01-01',
    end_date='2023-01-01',
    additional_filters=additional_filters,
    bowler_id='495d42a5')
print(query)
curs.execute(query)
for row in curs:
    print(row)


with base_data as (
  select * from delivery_records WHERE __time > '2017-01-01' AND __time < '2023-01-01'  AND "match_type" = 'T20'  and batter_id in ('ba607b88')
), dismissals as (
  select sum("wicket_sum") as dismissal_count from base_data WHERE "bowler_id" = '495d42a5'
)
SELECT
  sum("delivery_count")*1.0/(SELECT sum(dismissal_count) from dismissals) as strike_rate,
  sum("total_run_sum")*1.0/(SELECT sum(dismissal_count) from dismissals) as average
from base_data
where "bowler_id" = '495d42a5'

Row(strike_rate=30.0, average=35.0)


In [72]:
fielding_stats_template = Template("""
with base_data as (
  select * from delivery_records WHERE __time > '$start_date' AND __time < '$end_date'  AND "match_type" = 'T20' $additional_filters
)
SELECT sum(wicket_sum) as fielding_dismissals from base_data where "wicket_fielder_id"= '$fielder_id'
""")
additional_filters=''
query = fielding_stats_template.substitute(
    start_date = '2017-01-01',
    end_date='2023-01-01',
    additional_filters=additional_filters,
    fielder_id='ba607b88')
print(query)
curs.execute(query)
for row in curs:
    print(row)


with base_data as (
  select * from delivery_records WHERE __time > '2017-01-01' AND __time < '2023-01-01'  AND "match_type" = 'T20' 
)
SELECT sum(wicket_sum) as fielding_dismissals from base_data where "wicket_fielder_id"= 'ba607b88'

Row(fielding_dismissals=69)
